# PGN to NLT

We will create a all the possible way the user can ask the bot to play chess move. Convert that data into a text and convert text to audio for training the bot. 

This will helps us create enough dataset to train the NN.

### Pre-requisite
~~1. Run the extract code and convert the .zip into .json file~~


Set the number of audio data this will create 1000 audio files under training/audio and the file name will be the row of training.text file

In [132]:
training_rows_count = 1000

In [121]:
# Install gtts
# !pip3 install gtts

In [133]:
import os
import random
from gtts import gTTS
from concurrent.futures import ThreadPoolExecutor


In [123]:
# Helper functions

def create_audio_file(text, file_path, language='en'):
  tts = gTTS(text=text, lang=language, slow=False)
  tts.save(file_path)

def count_lines(file_path):
  print("running count...")
  try:
    with open(file_path, 'r') as file:
      lines = file.readlines()
      return len(lines)
  except FileNotFoundError:
    print(f"File '{file_path}' not found.")
    return 0
  
def remove_template(text):
  while "_" in text:
    text = text.replace("_from", random.choice(_from), 1)
    text = text.replace("_to", random.choice(_to), 1)
    text = text.replace("_random_rank2", random.choice(_random_rank2), 1)
    text = text.replace("_random_rank4", random.choice(_random_rank4), 1)
    text = text.replace("_random_rank5", random.choice(_random_rank5), 1)
    text = text.replace("_random_rank7", random.choice(_random_rank7), 1)
    text = text.replace("_random_piece", random.choice(_random_piece), 1)
    text = text.replace("_random_power_piece", random.choice(_random_power_piece), 1)
  return text

In [124]:
# Default variable
_random_rank2 = ["a2", "b2", "c2", "d2", "e2", "f2", "g2", "h2"]
_random_rank4 = ["a4", "b4", "c4", "d4", "e4", "f4", "g4", "h4"]
_random_rank5 = ["a5", "b5", "c5", "d5", "e5", "f5", "g5", "h5"]
_random_rank7 = ["a7", "b7", "c7", "d7", "e7", "f7", "g7", "h7"]

_random_piece = ["pawn", "knight", "bishop", "rook", "queen", "king"]
_random_power_piece = ["knight", "bishop", "rook", "queen", "king"]

_from = [ 
  "a1", "b1", "c1", "d1", "e1", "f1", "g1", "h1",
  "a2", "b2", "c2", "d2", "e2", "f2", "g2", "h2",
  "a3", "b3", "c3", "d3", "e3", "f3", "g3", "h3",
  "a4", "b4", "c4", "d4", "e4", "f4", "g4", "h4",
  "a5", "b5", "c5", "d5", "e5", "f5", "g5", "h5",
  "a6", "b6", "c6", "d6", "e6", "f6", "g6", "h6",
  "a7", "b7", "c7", "d7", "e7", "f7", "g7", "h7",
  "a8", "b8", "c8", "d8", "e8", "f8", "g8", "h8",
]

_to = [
  "a1", "b1", "c1", "d1", "e1", "f1", "g1", "h1",
  "a2", "b2", "c2", "d2", "e2", "f2", "g2", "h2",
  "a3", "b3", "c3", "d3", "e3", "f3", "g3", "h3",
  "a4", "b4", "c4", "d4", "e4", "f4", "g4", "h4",
  "a5", "b5", "c5", "d5", "e5", "f5", "g5", "h5",
  "a6", "b6", "c6", "d6", "e6", "f6", "g6", "h6",
  "a7", "b7", "c7", "d7", "e7", "f7", "g7", "h7",
  "a8", "b8", "c8", "d8", "e8", "f8", "g8", "h8",
]


In [125]:
template_training_data = [
  # by place
    "_from _to",
    "_from to _to",
    "from _from to _to",
    "move _from to _to",
    "_from capture _to",
    "capture _to",
    "_from takes _to",
    "takes _to",
  # pawn
    # pawn double move
    "_random_rank4", # Pawn move from e2 to e4
    "_random_rank5", # Pawn move from e7 to e5
    "pawn _random_rank4",
    "pawn _random_rank5",
    "_random_rank2 to _random_rank4",
    "_random_rank7 to _random_rank5",
    "pawn _random_rank2 to _random_rank4",
    "pawn _random_rank7 to _random_rank5",
    # promote only rank 7 and rank
    "pawn _random_rank7 promote _random_power_piece",
    "pawn _random_rank7 promote to _random_power_piece",
    "pawn _random_rank2 promote _random_power_piece",
    "pawn _random_rank2 promote to _random_power_piece",
  # by piece
    "move _random_piece _from _to",
    "move _random_piece _to",
    "move _random_piece _to",
    "_random_piece _from _to",
    "_random_piece _to",
    "_random_piece _to",
    "_random_piece _from to _to",
    "_random_piece to _to",
    "_random_piece to _to",
    # captures
    "capture _to _random_piece",
    "takes _to _random_piece",
    
    "_random_piece _from capture _to",
    "_random_piece capture _to",
    "_random_piece capture _to _random_piece",
    
    "_random_piece _from takes _to",
    "_random_piece takes _to",
    "_random_piece takes _to _random_piece",
]
  
# run this only 10% training data - less data is required
template_castle = [
  "long side castle",
  "king side castle",
  "castle",
  "short side castle",
  "queen side castle"
]

# en passant
template_en_passant = [
  "en passant _random_rank4",
  "en passant _random_rank5",
  "pawn en passant _random_rank4",
  "pawn en passant _random_rank5",
]


In [126]:
file_dir = "training"
audio_dir = f"{file_dir}/audio"
training_path = f"{file_dir}/training.txt"

def truncate_data(path):
  with open(path, 'w') as file:
    file.truncate()

def append_data(content):
  has_content = count_lines(training_path) != 0
  with open(training_path, 'a') as file:
    if has_content:
      content = '\n' + content
    file.write(content)
    has_content = True
  

# Create the directory if it doesn't exist
if os.path.exists(file_dir):
  truncate_data(training_path)
else:
  os.makedirs(file_dir)
  
  
directory = os.path.dirname(file_path)
# Create the directory if it doesn't exist
if os.path.exists(audio_dir) is not True:
  os.makedirs(audio_dir)

In [130]:
training_data = []

for i in range(training_rows_count):
  text = ""
  random_decimal = round(random.uniform(0, 1), 2)

  # 10% training data contains about castle
  if random_decimal < 0.2:
    text = random.choice(template_castle)
  # 10% training data contains about en passant
  elif random_decimal < 0.25:
    text = random.choice(template_en_passant)
  else:
    text = random.choice(template_training_data)
    
  if text:
    text = remove_template(text)
    training_data.append(text)


append_data('\n'.join(training_data))
print(f"File {file_path} created.")


running count...
File training/training.txt created.


In [134]:
for i in range(count_lines(training_path)):
  text = training_data[i]
  audio_file_path = f"{audio_dir}/{i}.mp3"
  with ThreadPoolExecutor() as executor:
    executor.map(create_audio_file, text, audio_file_path)
  

running count...


KeyboardInterrupt: 